In [1]:
!git clone https://github.com/Maharishiva/LiT-agent.git
%cd LiT-agent

Cloning into 'LiT-agent'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 154 (delta 15), reused 41 (delta 12), pack-reused 107 (from 1)
Receiving objects: 100% (154/154), 267.97 MiB | 49.89 MiB/s, done.
Resolving deltas: 100% (27/27), done.
/content/LiT-agent


In [2]:
!pip install -r requirements_colab.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 20.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of optax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 17.7 MB/s eta 0:00:00
   ━━━

In [1]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: maharishiva to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
cd LiT-agent/

/content/LiT-agent


In [6]:
!python train_PPO_trXL.py

2025-03-17 23:17:03.000356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742253423.022507    5506 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742253423.029164    5506 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Available devices: [CudaDevice(id=0)]
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: maharishiva to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.8
wandb: Run data is saved locally in /content/LiT-agent/wandb/run-20250317_231708-tdvtjxia
wandb: Run `wandb offline` to turn off 

10

[CudaDevice(id=0)]

# env replay

In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import imageio
from craftax.craftax.envs.craftax_symbolic_env import CraftaxSymbolicEnv
from craftax.craftax.renderer import render_craftax_pixels, render_craftax_symbolic
from transformerXL import Transformer
from trainer_PPO_trXL import ActorCriticTransformer

# Load saved model parameters and config
params = jnp.load("results_craftax/craftax/0_params.npy", allow_pickle=True).item()
config = jnp.load("results_craftax/craftax/0_config.npy", allow_pickle=True).item()

# Set up environment (assumed jitted) and its parameters
env = CraftaxSymbolicEnv()
env_params = env.default_params

# Create model
network = ActorCriticTransformer(
    action_dim=env.action_space(env_params).n,
    activation=config["ACTIVATION"],
    encoder_size=config["EMBED_SIZE"],
    hidden_layers=config["hidden_layers"],
    num_heads=config["num_heads"],
    qkv_features=config["qkv_features"],
    num_layers=config["num_layers"],
    gating=config["gating"],
    gating_bias=config["gating_bias"],
)

# Initialize random key and environment
rng = jax.random.PRNGKey(0)
rng, _rng = jax.random.split(rng)
obs, state = env.reset(_rng, env_params)

# Duplicate observation to create a dummy batch of size 2 (needed by the transformer)
obs = obs[None, :]                  
obs = jnp.concatenate([obs, obs], axis=0)

# Initialize memories and mask
memories = jnp.zeros((2, config["WINDOW_MEM"], config["num_layers"], config["EMBED_SIZE"]))
memories_mask = jnp.zeros((2, config["num_heads"], 1, config["WINDOW_MEM"] + 1), dtype=jnp.bool_)

# Define a single simulation step that is fully JIT-compilable.
@jax.jit
def sim_step(carry, _):
    rng, memories, obs, state = carry

    # Get model output (batched call)
    rng, _rng = jax.random.split(rng)
    pi, _, memories_out = network.apply(
        params, memories, obs, memories_mask, method=network.model_forward_eval
    )
    # Sample action from the first batch element (batch elements are identical)
    action = pi.sample(seed=_rng)[0]

    # Update memories: roll the time dimension and update the last slot.
    memories = jnp.roll(memories, -1, axis=1)
    new_mem = memories_out[0]  # shape: (num_layers, EMBED_SIZE)
    new_mem_batched = jnp.broadcast_to(new_mem, (2,) + new_mem.shape)
    memories = memories.at[:, -1].set(new_mem_batched)

    # Step the environment (jitted) and re-batch the observation
    rng, _rng = jax.random.split(rng)
    obs_unbatched, state, reward, done, info = env.step(_rng, state, action, env_params)
    obs = jnp.concatenate([obs_unbatched[None, :], obs_unbatched[None, :]], axis=0)

    new_carry = (rng, memories, obs, state)
    # For later rendering, we record the state at this step.
    output = state
    return new_carry, output

# Run the entire simulation loop with jax.lax.scan for 100 steps.
carry = (rng, memories, obs, state)
carry, state_history = jax.lax.scan(sim_step, carry, None, length=1000)

v_render = jax.vmap(lambda s: render_craftax_pixels(s, 64))
# This returns an array (or pytree of arrays) where the leading axis corresponds to the time steps.
frames_pixels = v_render(state_history)

# Convert the rendered frames from JAX to NumPy (if not already on the CPU).
frames_pixels_np = np.array(frames_pixels)

# Combine the initial frame with the rest.
# We assume that each frame is an image array.
frames = [frame for frame in frames_pixels_np]

# Save the frames as an MP4 video.
frames_np = np.array(frames[:], dtype=np.uint8)
imageio.mimsave('craftax_replay.mp4', frames_np, fps=30)
print(f"Saved {len(frames)} frames to craftax_replay.mp4")


Loading textures from cache.
Textures successfully loaded from cache.


ModuleNotFoundError: No module named 'distutils'